In [1]:
import json
import random
from collections import defaultdict
from itertools import chain
from typing import Iterable

import keras
import numpy
import pandas
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, StratifiedKFold

Using TensorFlow backend.


In [47]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scoring = 'accuracy'

In [80]:
# Copy-pasted from the game.
# https://heroes.cdnvideo.ru/vk/v0326/locale/ru.json.gz

lib = {
  "0": "Карен",
  "1": "Аврора",
  "10": "Безликий",
  "11": "Чабба",
  "12": "Арахна",
  "13": "Орион",
  "14": "Фокс",
  "15": "Джинджер",
  "16": "Данте",
  "17": "Моджо",
  "18": "Судья",
  "19": "Темная Звезда",
  "2": "Галахад",
  "20": "Артемис",
  "21": "Маркус",
  "22": "Пеппи",
  "23": "Лиэн",
  "24": "Тесак",
  "25": "Исмаил",
  "26": "Лилит",
  "27": "Лютер",
  "28": "Цин Мао",
  "29": "Дориан",
  "3": "Кира",
  "30": "Корнелиус",
  "31": "Джет",
  "32": "Гелиос",
  "33": "Ларс",
  "34": "Криста",
  "35": "Йорген",
  "36": "Майя",
  "37": "Джу",
  "38": "Эльмир",
  "39": "Зири",
  "4": "Астарот",
  "40": "Небула",
  "41": "К'арх",
  "5": "Кай",
  "6": "Фобос",
  "7": "Тея",
  "8": "Сорвиголова",
  "9": "Хайди",
}

In [83]:
# Parse battle logs.

def parse_heroes(heroes: Iterable[dict], sign: int, result: dict):
    for hero in heroes:
        for hero_key in ('Level', 'Color', 'Star'):
            result[f'''{lib[str(hero['id'])]} {hero_key}'''] += sign * hero[hero_key.lower()]


def parse_battle(line: str) -> dict:
    battle = json.loads(line)
    result = defaultdict(int)

    parse_heroes(battle.get('attackers') or battle['player'], +1, result)
    parse_heroes(battle.get('defenders') or battle['enemies'], -1, result)
        
    return {'Win': battle['win'], **result}

In [84]:
# Load the logs into a data frame.

def invert_column(series: pandas.Series):
    """
    Inverts the column to make an "opposite" battle.
    """
    return series == False if series.name == 'Win' else -series

lines = list(chain.from_iterable([
    open('battles.jsonl'),
    open('battles-lilia.jsonl'),
]))

battles = pandas.DataFrame([parse_battle(line) for line in lines]).fillna(value=0)
battles.drop_duplicates(inplace=True)  # because the files may contain duplicate battles
battles = pandas.concat((battles, battles.apply(invert_column)))  # add inverted battles
battles.head()

,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
0,False,0.0,0.0,0.0,4.0,35.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,0.0,0.0,0.0,-2.0,-10.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,0.0,0.0,0.0,4.0,36.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-45.0,-2.0
3,False,0.0,0.0,0.0,-1.0,-10.0,-1.0,-4.0,-46.0,-3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.0,0.0,0.0,-1.0,-4.0,-2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
battles.describe()

,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,Артемис Level,Артемис Star,Астарот Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
count,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000,...,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000,352.000000
mean,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
std,2.451815,19.377198,0.867258,3.668609,29.195578,1.786304,3.483784,29.127483,1.281025,5.200263,...,0.311233,1.360828,11.410996,0.413449,1.422249,11.459081,0.599145,0.951831,7.734556,0.420283
min,-9.000000,-70.000000,-3.000000,-9.000000,-71.000000,-6.000000,-8.000000,-69.000000,-3.000000,-9.000000,...,-2.000000,-8.000000,-71.000000,-3.000000,-8.000000,-70.000000,-3.000000,-7.000000,-60.000000,-3.000000
25%,-0.000000,-0.000000,-0.000000,0.000000,-0.250000,-0.000000,-0.000000,-0.000000,-0.000000,-6.000000,...,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,6.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,9.000000,70.000000,3.000000,9.000000,71.000000,6.000000,8.000000,69.000000,3.000000,9.000000,...,2.000000,8.000000,71.000000,3.000000,8.000000,70.000000,3.000000,7.000000,60.000000,3.000000


In [86]:
x = battles.drop(['Win'], axis=1)
y = battles['Win']

In [87]:
classifier = LogisticRegression(max_iter=1000, fit_intercept=False)
param_grid = {'C': numpy.logspace(-6, 2, num=1000)}

numpy.random.seed(42)
grid_search = GridSearchCV(classifier, param_grid, cv=cv, scoring=scoring, n_jobs=4).fit(x, y)

print(f'Best score: {grid_search.best_score_}')
print(f'Best params: {grid_search.best_params_}')
print(f'Classes: {grid_search.best_estimator_.classes_}')

scores = cross_val_score(grid_search.best_estimator_, x, y, scoring=scoring, cv=cv)
print(f'CV score: {scores.mean()} (std: {scores.std()})')

Best score: 0.9715909090909091
Best params: {'C': 28.540097698292374}
Classes: [False  True]
CV score: 0.9718954248366014 (std: 0.03540740713929898)


In [88]:
pandas.DataFrame({'Feature': x.columns, 'Importance': grid_search.best_estimator_.coef_[0]}) \
    .set_index('Feature') \
    .sort_values('Importance', ascending=False) \
    .head(n=20)

,Importance
Feature,
Астарот Star,9.281038
Зири Color,8.040298
Фобос Color,6.202511
Джинджер Color,5.709298
Судья Star,5.300113
Аврора Color,4.897856
Судья Color,4.826681
Фобос Star,3.379874
Арахна Color,3.301794


In [89]:
result = pandas.concat((
    pandas.Series(grid_search.best_estimator_.predict(x), index=battles.index, name='Predicted'),
    pandas.Series(grid_search.best_estimator_.predict_proba(x)[:, 1], index=battles.index, name='Probability'),
    battles,
), axis=1)
result['Probability'] = result['Probability'].apply('{:.2f}'.format)
result.head()

,Predicted,Probability,Win,Аврора Color,Аврора Level,Аврора Star,Арахна Color,Арахна Level,Арахна Star,Артемис Color,...,Хайди Star,Цин Мао Color,Цин Мао Level,Цин Мао Star,Чабба Color,Чабба Level,Чабба Star,Эльмир Color,Эльмир Level,Эльмир Star
0,False,0.00,False,0.0,0.0,0.0,4.0,35.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,0.00,False,0.0,0.0,0.0,-2.0,-10.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,0.01,False,0.0,0.0,0.0,4.0,36.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-5.0,-45.0,-2.0
3,False,0.03,False,0.0,0.0,0.0,-1.0,-10.0,-1.0,-4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,0.03,False,0.0,0.0,0.0,-1.0,-4.0,-2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:
# Experimental NN model.
# I want it to catch an interaction between different heroes (e.g. Йорген is good in conjunction with Исмаил).

def create_model() -> keras.Model:
    model = Sequential()
    model.add(Dense(1000, input_dim=x.columns.size, activation='sigmoid'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(1, input_dim=x.columns.size, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

keras_classifier = KerasClassifier(build_fn=create_model, epochs=100, verbose=0)

numpy.random.seed(42)
scores = cross_val_score(keras_classifier, x, y, cv=cv)

print(f'Score: {scores.mean()} (std: {scores.std()})')

Score: 0.977124183006536 (std: 0.038445597829680385)
